# Scraping, Inserting, and Aggregating Hockey Data

This notebook demonstrates how to scrape player data from Spotrac, insert the scraped data into a database, and then aggregate the data for further analysis.

## 1. Setup and Environment Configuration



In [ ]:
# Import necessary libraries
import os

import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from dotenv import load_dotenv
from sqlalchemy import (
    BigInteger,
    Column,
    Float,
    Integer,
    MetaData,
    String,
    Table,
    create_engine,
)
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.orm import sessionmaker

# Load environment variables
load_dotenv()

# Set up the database connection
DATABASE_TYPE = os.getenv("DATABASE_TYPE")
DBAPI = os.getenv("DBAPI")
ENDPOINT = os.getenv("ENDPOINT")
USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWORD")
PORT = int(os.getenv("PORT", 5432))
DATABASE = os.getenv("DATABASE")

# Create the connection string
connection_string = f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)


## 2. Scraping Player Data from Spotrac

We will scrape player cap hit data for the years 2015, 2016, and 2017 from the Spotrac website using Selenium and BeautifulSoup.
The selenium driver is set to safari browser. If you are using a different browser, you will need to adjust the code. 
driver = webdriver.Safari()

In [ ]:
# Set up the Safari WebDriver
driver = webdriver.Safari()

# Base URL to scrape
BASE_URL = "https://www.spotrac.com/nhl/rankings/player/_/year/{}/sort/cap_total"

# Years to scrape
years = [2015, 2016, 2017]

# Directory to store CSV files
OUTPUT_DIR = "player_cap_hits"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Dictionary to store DataFrames for each year
dfs_by_year = {}

def split_player_name(name):
    """Function to clean and split player name"""
    name_parts = name.split()
    first_name = name_parts[0]
    last_name = " ".join(name_parts[1:]) if len(name_parts) > 1 else ""
    return first_name, last_name

# Loop through each year and scrape the data
for year in years:
    url = BASE_URL.format(year)
    driver.get(url)

    # Wait until the table is loaded
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "list-group-item"))
    )

    # Scroll to the bottom of the page to load all content (if applicable)
    while True:
        previous_height = driver.execute_script("return document.body.scrollHeight")
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(2)  # Wait for new data to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == previous_height:
            break  # Exit the loop when no more new content is loaded

    # Get page source and parse with BeautifulSoup
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # Find the elements containing the player names and cap hits
    first_names = []
    last_names = []
    cap_hits = []

    for item in soup.find_all("li", class_="list-group-item"):
        name_div = item.find("div", class_="link")
        cap_hit_span = item.find("span", class_="medium")
        if name_div and cap_hit_span:
            name = name_div.text.strip()
            first_name, last_name = split_player_name(name)
            cap_hit = cap_hit_span.text.strip()
            first_names.append(first_name)
            last_names.append(last_name)
            cap_hits.append(cap_hit)

    # Create a DataFrame to store the results
    df = pd.DataFrame(
        {"firstName": first_names, "lastName": last_names, "capHit": cap_hits}
    )

    # Store the DataFrame in the dictionary
    dfs_by_year[year] = df

# Close the driver
driver.quit()

# Display the DataFrames for each year
for year, df in dfs_by_year.items():
    print(f"Data for {year}:")
    display(df.head())

# Save the DataFrames to CSV files
for year, df in dfs_by_year.items():
    csv_path = os.path.join(OUTPUT_DIR, f"player_cap_hits_{year}.csv")
    df.to_csv(csv_path, index=False, mode="w")

print(f"Data saved to {OUTPUT_DIR} directory.")


## 3. Inserting Scraped Data into the Database

Next, we will insert the scraped data from the CSV files into the `hockey_stats` database.


In [ ]:
# Define function to create cap hit tables
def create_caphit_table(table_name):
    """Define table creation function to avoid repetition"""
    return Table(
        table_name,
        metadata,
        Column("firstName", String(50)),
        Column("lastName", String(50)),
        Column("capHit", String(50)),
    )

# Create tables for each season
metadata = MetaData()
seasons = ["20152016", "20162017", "20172018"]
tables = {season: create_caphit_table(f"player_cap_hit_{season}") for season in seasons}
metadata.create_all(engine)

Session = sessionmaker(bind=engine)

def insert_data_from_csv(engine, table_name, file_path):
    """Insert data from CSV into the specified database table."""
    try:
        df = pd.read_csv(file_path)
        df.to_sql(table_name, con=engine, if_exists="replace", index=False)
        print(f"Data inserted successfully into {table_name}")

        # Remove the file after successful insertion
        os.remove(file_path)
        print(f"File {file_path} deleted successfully.")

    except SQLAlchemyError as e:
        print(f"Error inserting data into {table_name}: {e}")
    except FileNotFoundError as e:
        print(f"File not found: {file_path} - {e}")
    except Exception as e:
        print(f"Error occurred while processing file '{file_path}': {e}")

# Define directories and mappings for insertion
csv_files_and_mappings = [
    ("player_cap_hits/player_cap_hits_2015.csv", "player_cap_hit_20152016"),
    ("player_cap_hits/player_cap_hits_2016.csv", "player_cap_hit_20162017"),
    ("player_cap_hits/player_cap_hits_2017.csv", "player_cap_hit_20172018"),
]

# Insert data into database tables
with Session() as session:
    for file_path, table_name in csv_files_and_mappings:
        insert_data_from_csv(engine, table_name, file_path)

    print("Data inserted successfully into all tables.")


## 4. Aggregate Data and insert into the Database

Finally, we will aggregate the data per season per player and insert the results into an aggregated table in the database.


In [ ]:
# Define function to get data from the database
def get_data_from_db(query):
    """Function to get data from the database."""
    with engine.connect() as connection:
        return pd.read_sql(query, connection)

def create_aggregated_table(table_name):
    """Create table schema for aggregated table."""
    metadata = MetaData()
    Table(
        table_name,
        metadata,
        Column("player_id", BigInteger, primary_key=True),
        Column("firstName", String),
        Column("lastName", String),
        Column("corsi_for", Float),
        Column("corsi_against", Float),
        Column("corsi", Float),
        Column("CF_Percent", Float),
        Column("timeOnIce", Float),
        Column("game_count", Integer),
        Column("Cap_Hit", Float),
    )
    metadata.create_all(engine)

# Loop through each season to aggregate data
for season in ["20152016", "20162017", "20172018"]:
    CORSI_QUERY = f"SELECT * FROM raw_corsi_{season}"
    df_corsi = get_data_from_db(CORSI_QUERY)
    if "Unnamed: 0" in df_corsi.columns:
        df_corsi = df_corsi.drop(columns=["Unnamed: 0"])

    GSS_TOI_QUERY = 'SELECT game_id, player_id, "timeOnIce" FROM game_skater_stats'
    df_gss_toi = get_data_from_db(GSS_TOI_QUERY)

    PLAYER_INFO_QUERY = (
        'SELECT player_id, "firstName", "lastName", "primaryPosition" FROM player_info'
    )
    df_player_info = get_data_from_db(PLAYER_INFO_QUERY)

    # Merge dataframes
    df_all = pd.merge(df_corsi, df_gss_toi, on=["game_id", "player_id"])
    df_all = pd.merge(df_all, df_player_info, on="player_id")

    # Group and aggregate player stats
    df_grouped_all = (
        df_all.groupby("player_id")
        .agg(
            {
                "firstName": "first",
                "lastName": "first",
                "corsi_for": "mean",
                "corsi_against": "mean",
                "corsi": "mean",
                "CF_Percent": "mean",
                "timeOnIce": "mean",
                "game_id": "count",
            }
        )
        .reset_index()
        .rename(columns={"game_id": "game_count"})
    )

    PLAYER_SALARY_QUERY = (
        f'SELECT "firstName", "lastName", "capHit" FROM player_cap_hit_{season}'
    )
    df_player_salary = get_data_from_db(PLAYER_SALARY_QUERY)
    print(df_player_salary.head())

    # Convert capHit from string to float
    df_player_salary["capHit"] = (
        df_player_salary["capHit"].replace(r"[\$,]", "", regex=True).astype(float)
    )

    # Merge aggregated stats with salary info
    df_grouped_all = pd.merge(
        df_grouped_all, df_player_salary, on=["firstName", "lastName"]
    )

    # Post-processing
    df_grouped_all["CF_Percent"] = (df_grouped_all["CF_Percent"].round(4) * 100).round(4)
    df_grouped_all["timeOnIce"] = df_grouped_all["timeOnIce"].round(2)

    THRESHOLD = 82 * 0.32
    df_grouped_all = df_grouped_all.query(f"game_count >= {THRESHOLD}")

    df_grouped_all["CF_Percent"] = df_grouped_all["CF_Percent"].apply(
        lambda x: np.round(x, 4)
    )
    df_grouped_all["timeOnIce"] = df_grouped_all["timeOnIce"].apply(
        lambda x: np.round(x, 2)
    )

    df_grouped_all = df_grouped_all.sort_values("CF_Percent", ascending=False)

    aggregated_table_name = f"aggregated_corsi_{season}"
    create_aggregated_table(aggregated_table_name)

    # Insert aggregated data into the new table
    df_grouped_all.to_sql(
        aggregated_table_name, con=engine, if_exists="replace", index=False
    )

    print(f"Data inserted successfully into {aggregated_table_name}")


Make another database table that will be used for graphics.

In [ ]:
import os
import time

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# Set up the Safari WebDriver
driver = webdriver.Safari()

# Base URL to scrape
BASE_URL = "https://www.spotrac.com/nhl/cap/_/year/{}/sort/cap_maximum_space2"

# Years to scrape
years = [2015, 2016, 2017]

# Directory to store CSV files
OUTPUT_DIR = "team_salaries"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Loop through each year and scrape the data
for year in years:
    url = BASE_URL.format(year)
    driver.get(url)

    # Wait until the table is loaded
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "tablesorter-headerRow"))
    )

    # Scroll to the bottom of the page to load all content (if applicable)
    while True:
        previous_height = driver.execute_script("return document.body.scrollHeight")
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(2)  # Wait for new data to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == previous_height:
            break  # Exit the loop when no more new content is loaded

    # Get page source and parse with BeautifulSoup
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # Find all rows in the table body
    rows = soup.find("tbody").find_all("tr")

    # Find the elements containing the team name and total player payroll
    team_names = []
    team_payroll = []

    # Iterate over each row to extract the name and payroll value
    for row in rows:
        # Get the team name, usually from the 'a' tag within the second 'td' element
        team_name_tag = row.find_all("td")[1].find("a")
        if team_name_tag:
            team_name = team_name_tag.get_text(strip=True)
        else:
            team_name = row.find_all("td")[1].get_text(strip=True)

        # Get the correct payroll column (assumed to be in the 7th column)
        payroll_column = row.find_all("td")[6].get_text(strip=True)

        team_names.append(team_name)
        team_payroll.append(payroll_column)

    # Create a DataFrame to store the results
    df = pd.DataFrame({"Team": team_names, "Total_Payroll": team_payroll})

    # Save DataFrame to a CSV file
    output_file = os.path.join(OUTPUT_DIR, f"team_salary_{year}.csv")
    df.to_csv(output_file, index=False)

# Close the driver
driver.quit()

print("Scraping completed. CSV files saved in the 'output' directory.")

In [ ]:
import os

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.safari.service import Service as SafariService
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException 

# Directory to store CSV files
OUTPUT_DIR = "team_records"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Set up the Selenium WebDriver for Safari
service = SafariService()
driver = webdriver.Safari(service=service)

# Base URL to scrape
BASE_URL = "https://www.hockey-reference.com/leagues/NHL_{}.html"

# Years to scrape
years = [2016, 2017, 2018]

# Columns to extract
columns_to_extract = {
    "team_name": "Team",
    "games": "GP",
    "wins": "W",
    "losses": "L",
    "losses_ot": "OTL",
    "points": "PTS",
}

# XPath to locate specific columns in the table
xpaths = {
    "team_name": './/td[@data-stat="team_name"]/a',
    "games": './/td[@data-stat="games"]',
    "wins": './/td[@data-stat="wins"]',
    "losses": './/td[@data-stat="losses"]',
    "losses_ot": './/td[@data-stat="losses_ot"]',
    "points": './/td[@data-stat="points"]',
}

# Step 2: Create a dictionary mapping full names to abbreviations and team IDs
name_to_abbreviation = {
    "Washington Capitals": ("WAS", 15),
    "Dallas Stars": ("DAL", 25),
    "St. Louis Blues": ("STL", 19),
    "Pittsburgh Penguins": ("PIT", 5),
    "Chicago Blackhawks": ("CHI", 16),
    "Florida Panthers": ("FLA", 13),
    "Anaheim Ducks": ("ANA", 24),
    "Los Angeles Kings": ("LAK", 26),
    "New York Rangers": ("NYR", 3),
    "New York Islanders": ("NYI", 2),
    "San Jose Sharks": ("SJS", 28),
    "Tampa Bay Lightning": ("TBL", 14),
    "Nashville Predators": ("NSH", 18),
    "Philadelphia Flyers": ("PHI", 4),
    "Detroit Red Wings": ("DET", 17),
    "Boston Bruins": ("BOS", 6),
    "Minnesota Wild": ("MIN", 30),
    "Carolina Hurricanes": ("CAR", 12),
    "Ottawa Senators": ("OTT", 9),
    "New Jersey Devils": ("NJD", 1),
    "Montreal Canadiens": ("MTL", 8),
    "Colorado Avalanche": ("COL", 21),
    "Buffalo Sabres": ("BUF", 7),
    "Winnipeg Jets": ("WPG", 52),
    "Arizona Coyotes": ("ARI", 53),
    "Calgary Flames": ("CGY", 20),
    "Columbus Blue Jackets": ("CBJ", 29),
    "Vancouver Canucks": ("VAN", 23),
    "Edmonton Oilers": ("EDM", 22),
    "Toronto Maple Leafs": ("TOR", 10),
    "Vegas Golden Knights": ("VGK", 54)
}

# Convert the dictionary into a DataFrame
team_df = pd.DataFrame.from_dict(
    name_to_abbreviation, orient="index", columns=["Abbreviation", "Team_ID"]
)
team_df.reset_index(inplace=True)
team_df.rename(columns={"index": "Team_Name"}, inplace=True)

# Display the team DataFrame
print("Team DataFrame:")
print(team_df)

# Loop through each year and scrape the data
for year in years:
    # Generate the URL for the current year
    url = BASE_URL.format(year)

    # Navigate to the URL
    driver.get(url)

    # Wait until the table is loaded
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="stats"]/tbody/tr'))
    )

    # Locate the rows in the table
    rows = driver.find_elements(By.XPATH, '//*[@id="stats"]/tbody/tr')

    # Prepare the list to hold the data
    data = []

        # Iterate over each row and extract the specific columns
    for row in rows:
        team_data = {}
        for key, xpath in xpaths.items():
            try:
                element = row.find_element(By.XPATH, xpath)
                team_data[columns_to_extract[key]] = element.text
            except NoSuchElementException:
                # If the element is not found, log it and move on
                print(f"Element '{columns_to_extract[key]}' not found for team in year {year}.")
                team_data[columns_to_extract[key]] = "N/A"  # Or use another placeholder value
        data.append(team_data)

    # Convert the data to a DataFrame
    df = pd.DataFrame(data, columns=columns_to_extract.values())

    # Merge the scraped data with the team_df to include Abbreviation and Team_ID
    merged_df = pd.merge(df, team_df, left_on="Team", right_on="Team_Name", how="left")

    # Drop the redundant 'Team_Name' column from the merged DataFrame
    merged_df.drop(columns=["Team_Name"], inplace=True)

    # Save the merged DataFrame to a CSV file for the current year
    output_file = os.path.join(OUTPUT_DIR, f"NHL_{year}_team_stats.csv")
    merged_df.to_csv(output_file, index=False)

    print(f"Scraping and merging completed for {year}. Data saved to '{output_file}'")

# Close the Safari WebDriver
driver.quit()

print("All seasons scraped, merged, and saved.")


In [ ]:
import os

import pandas as pd
from sqlalchemy import (
    MetaData,
)
from sqlalchemy.orm import sessionmaker

from db_utils import get_db_engine, get_metadata

engine = get_db_engine()
metadata = get_metadata()

# Define metadata and tables
metadata = MetaData()

# Create tables for each season
seasons = ["2016", "2017", "2018"]
# tables = {season: create_team_data_table(f"team_data_{season}") for season in seasons}

# Create tables in the database
metadata.create_all(engine)

Session = sessionmaker(bind=engine)

for season in seasons:
    try:
        # Define the paths to the CSV files for each season
        stats_path = f"team_records/NHL_{int(season)}_team_stats.csv"
        salary_path = f"team_salaries/team_salary_{int(season) - 1}.csv"

        # Load the stats and team salary data
        stats_data = pd.read_csv(stats_path)
        salary_data = pd.read_csv(salary_path)

        # Merge on different column names
        merged_data = pd.merge(
            stats_data,
            salary_data,
            left_on="Abbreviation",
            right_on="Team",
            how="inner",
        )

        # Drop the redundant 'Team_y' column from the merged DataFrame
        merged_data.drop(columns=["Team_y"], inplace=True)

        # Display the merged data to verify
        print(f"Merged data for season {season}:\n", merged_data.head())

        # Insert the merged data into a new table in the database
        table_name = f"merged_team_stats_{season}"
        merged_data.to_sql(table_name, engine, if_exists="replace", index=False)
        print(
            f"Data for season {season} has been successfully inserted into the database."
        )

        # Delete the CSV files after successful insertion
        try:
            os.remove(stats_path)
            os.remove(salary_path)
            print(f"CSV files for season {season} have been successfully deleted.")
        except OSError as e:
            print(f"Error: {e.strerror} - while deleting files for season {season}")
    except Exception as e:
        print(f"Failed to process data for season {season}. Error: {e}")

## 5. Conclusion

In this notebook, we scraped player data from Spotrac, inserted the data into a database, and aggregated it for further analysis. These steps are crucial for generating meaningful insights into player performance and salary metrics.
